In [1]:
from playwright.async_api import async_playwright
import asyncio
import time

In [2]:
browser = None
page = None

In [3]:
async def automate_booking(session_data):
    """
    Automates login, search train, and prepares up to captcha
    Returns the capthsa image path
    """

    global browser, page
    p = await async_playwright().start()
    browser = await p.chromium.launch(headless=False)
    page = await browser.new_page()

    await page.goto("https://www.irctc.co.in")
    await page.wait_for_timeout(3000)

    await page.click("text=LOGIN")
    await page.wait_for_timeout(2000)

    await page.fill("input[formcontrolname='userId']", session_data['credentials']['username'])
    await page.fill("input[formcontrolname='password']", session_data['credentials']['password'])

    
    captcha_path = f"captcha_{session_data['credentials']['username']}.png"
    captcha_element = await page.locator("img.captcha-img").screenshot(path=captcha_path)

    return "captcha.png"


In [4]:
async def complete_booking(session_data):
    """
    Completes the booking process after receiving captcha from the user.
    """
    global browser, page

    # Fill Captcha
    await page.fill("input[formcontrolname='captchaValue']", session_data['captcha'])
    await page.click("button[label='Sign In']")
    await page.wait_for_timeout(5000)

    # Search Trains
    await page.fill("input[placeholder='From*']", session_data['from'])
    await page.fill("input[placeholder='To*']", session_data['to'])
    await page.fill("input[placeholder='Journey Date(dd-mm-yyyy)*']", session_data['date'])
    await page.click("button[label='Find Trains']")
    await page.wait_for_timeout(5000)

    # Select Train and Class
    await page.locator(f"text={session_data['train_type']}").click()
    await page.locator(f"text={session_data['class']}").click()

    # Book Ticket
    await page.click("button[label='Book Now']")
    await page.wait_for_timeout(3000)

    # Fill Passenger Details
    for idx, (name, age, gender) in enumerate(session_data['passengers']):
        await page.fill(f"input[formcontrolname='passengerName{idx}']", name)
        await page.fill(f"input[formcontrolname='passengerAge{idx}']", age)
        await page.select_option(f"select[formcontrolname='passengerGender{idx}']", gender)

    # Proceed to Payment Page
    await page.click("button[label='Continue Booking']")
    await page.wait_for_timeout(5000)

    # Note: Payment must be done manually
    # Download Ticket PDF (post-payment step)
    ticket_pdf_path = f"ticket_{session_data['credentials']['username']}.pdf"
    await page.click("button[label='My Bookings']")
    await page.wait_for_timeout(3000)
    await page.locator("text=Download Ticket").click()

    return ticket_pdf_path

In [5]:
# session_data = {
#     'credentials': {
#         'username': 'Gopumanvith',
#         'password': 'Gopu@123'
#     }
# }

# captcha_path = await automate_booking(session_data= session_data)

# print(f'Captch image saved at {captcha_path}')

In [1]:
import os
import playwright

In [ ]:
chrome_executable_path = "C:\\Users\\saish\\Downloads\\chrome-win64\\chrome.exe"
os.system(f'"{chrome_executable_path}" --remote-debugging-port=9222')

# Then connect via Playwright:
browser = await playwright.chromium.connect_over_cdp("http://localhost:9222")